In [4]:
from datetime import datetime, timedelta

# pandas and plotting libraries for visualizations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# module containing functions for manipulation pyspark dataframes
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.window import Window

# class which will let us create spark objects
from pyspark.sql import SparkSession

# helper functions for the class
from helpers import display, read_df, write_df

## [PySpark SQL docs](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)

## Create a Spark Session

In [2]:
spark = (
    SparkSession
    .builder
    .appName('data_processing')
    .master('local[2]')
    .getOrCreate()
)

## Read in data file

In [5]:
df = read_df(spark, '../taxi_2016.parquet')

In [6]:
df.columns

['trip_id',
 'taxi_id',
 'start_time',
 'end_time',
 'trip_miles',
 'pickup_census_tract',
 'dropoff_census_tract',
 'fare',
 'tips',
 'trip_total',
 'payment_type',
 'company']

In [7]:
# only have tip data for credit card transactions
tips = df.where(f.col('payment_type') == 'Credit Card')

In [9]:
tips_by_census_tract = tips.groupby('dropoff_census_tract').agg(f.avg(f.col('trip_miles')).alias('avg_miles'))
%time display(tips.join(tips_by_census_tract, 'dropoff_census_tract', 'left'), 10)

CPU times: user 20 ms, sys: 20 ms, total: 40 ms
Wall time: 18.7 s


,dropoff_census_tract,trip_id,taxi_id,start_time,end_time,trip_miles,pickup_census_tract,fare,tips,trip_total,payment_type,company,avg_miles
0,17031832600,2d495c6e8ee5cad52f64de2844f5bdd9999c29f5,75dd278c61cc4b3651b76a89407986db8362b1fa6b3947...,2016-07-06 18:45:00,2016-07-06 19:00:00,3.3,17031839100,13.25,2.06,15.81,Credit Card,None,3.349968
1,17031832600,2d4c21df0917fd4c630c704b4d1b501c8aec5e14,99ab99695f31d67888669dcd596cbd0cda61f5c717b8d7...,2016-12-16 19:15:00,2016-12-16 19:30:00,2.1,17031081800,9.50,1.50,11.50,Credit Card,None,3.349968
2,17031832600,2d4fbb3ea94d4ed30fb45c9c7f278ac75b632fff,6848fb65d081cae1ae77f1828f6ec5fce4bb75c28a81bc...,2016-10-26 16:00:00,2016-10-26 16:15:00,0.1,17031081800,12.25,3.05,15.30,Credit Card,Taxi Affiliation Services,3.349968
3,17031832600,2d50340947f1362cdebc258576e8c84423e25f6e,b07a41c369d99e7e7638e698063efe74c8cd8c25e3e9e8...,2016-01-21 20:00:00,2016-01-21 20:00:00,2.7,17031081600,10.25,2.55,12.80,Credit Card,Northwest Management LLC,3.349968
4,17031832600,2d51c476faa582cfa6a4ff99a1e8c08ac2396a0b,9d20d7617e35a8d763ca0bbe3b47308e4bb8e368a8f3bd...,2016-08-12 15:45:00,2016-08-12 17:00:00,15.0,17031980000,43.50,14.25,61.75,Credit Card,Northwest Management LLC,3.349968
5,17031832600,2d527e75c7f3ca5e1abd1618d6715b2c4a9128ab,791d74df896226a452a8e223e2ec9fa0df7d80bb7ca180...,2016-12-18 03:30:00,2016-12-18 03:45:00,2.7,17031081700,9.75,2.00,11.75,Credit Card,Taxi Affiliation Services,3.349968
6,17031832600,2d53ad1ad5ba8dae3763c4fc8ca12b5c0f8ef171,155ffe17bc32e7c3bfdfdba2750e82c6a23dab24f77d6c...,2016-02-27 21:00:00,2016-02-27 21:00:00,0.6,17031071300,4.75,1.00,5.75,Credit Card,None,3.349968
7,17031832600,2d544df4ac9b89d61a8231d88ebdcf7e8162f7ae,5cc8bfcc9c086a9271655b2a81d619ab5c9c4b3e75e40a...,2016-10-02 03:45:00,2016-10-02 03:45:00,2.6,17031062100,10.25,2.45,14.70,Credit Card,None,3.349968
8,17031832600,2d557486f1ebe80551492915d726bef7393ca91d,23f128209b227dfd806428d903d59e15ec4bb57458648d...,2016-03-05 02:45:00,2016-03-05 02:45:00,0.0,17031080300,6.75,2.00,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,3.349968
9,17031832600,2d5cb8290f83470a9fc8da34ac2f10ba2161367b,96a3f43a4eb7400d581a682773569e457cdcba002adf40...,2016-03-14 19:30:00,2016-03-14 20:00:00,4.2,17031320100,16.25,3.25,19.50,Credit Card,Dispatch Taxi Affiliation,3.349968


In [10]:
display(tips.groupby('dropoff_census_tract').agg(f.avg(f.col('tips')).alias('avg_tip')), 10)

,dropoff_census_tract,avg_tip
0,17031832600,3.070037
1,17031837400,3.575000
2,17031062200,3.500545
3,17031843200,2.444706
4,17031806900,6.286667
5,17031020602,2.136563
6,17031130200,1.000000
7,17031241300,3.069375
8,17031150501,2.000000
9,17031838000,2.888571


In [11]:
census_block_window = Window().partitionBy('dropoff_census_tract')
tips = tips.withColumn('avg_miles', f.avg(f.col('trip_miles')).over(census_block_window))
%time display(tips)

CPU times: user 30 ms, sys: 10 ms, total: 40 ms
Wall time: 10.9 s


,trip_id,taxi_id,start_time,end_time,trip_miles,pickup_census_tract,dropoff_census_tract,fare,tips,trip_total,payment_type,company,avg_miles
0,2d495c6e8ee5cad52f64de2844f5bdd9999c29f5,75dd278c61cc4b3651b76a89407986db8362b1fa6b3947...,2016-07-06 18:45:00,2016-07-06 19:00:00,3.30,17031839100,17031832600,13.25,2.06,15.81,Credit Card,None,3.349968
1,2d4c21df0917fd4c630c704b4d1b501c8aec5e14,99ab99695f31d67888669dcd596cbd0cda61f5c717b8d7...,2016-12-16 19:15:00,2016-12-16 19:30:00,2.10,17031081800,17031832600,9.50,1.50,11.50,Credit Card,None,3.349968
2,2d4fbb3ea94d4ed30fb45c9c7f278ac75b632fff,6848fb65d081cae1ae77f1828f6ec5fce4bb75c28a81bc...,2016-10-26 16:00:00,2016-10-26 16:15:00,0.10,17031081800,17031832600,12.25,3.05,15.30,Credit Card,Taxi Affiliation Services,3.349968
3,2d50340947f1362cdebc258576e8c84423e25f6e,b07a41c369d99e7e7638e698063efe74c8cd8c25e3e9e8...,2016-01-21 20:00:00,2016-01-21 20:00:00,2.70,17031081600,17031832600,10.25,2.55,12.80,Credit Card,Northwest Management LLC,3.349968
4,2d51c476faa582cfa6a4ff99a1e8c08ac2396a0b,9d20d7617e35a8d763ca0bbe3b47308e4bb8e368a8f3bd...,2016-08-12 15:45:00,2016-08-12 17:00:00,15.00,17031980000,17031832600,43.50,14.25,61.75,Credit Card,Northwest Management LLC,3.349968
5,2d527e75c7f3ca5e1abd1618d6715b2c4a9128ab,791d74df896226a452a8e223e2ec9fa0df7d80bb7ca180...,2016-12-18 03:30:00,2016-12-18 03:45:00,2.70,17031081700,17031832600,9.75,2.00,11.75,Credit Card,Taxi Affiliation Services,3.349968
6,2d53ad1ad5ba8dae3763c4fc8ca12b5c0f8ef171,155ffe17bc32e7c3bfdfdba2750e82c6a23dab24f77d6c...,2016-02-27 21:00:00,2016-02-27 21:00:00,0.60,17031071300,17031832600,4.75,1.00,5.75,Credit Card,None,3.349968
7,2d544df4ac9b89d61a8231d88ebdcf7e8162f7ae,5cc8bfcc9c086a9271655b2a81d619ab5c9c4b3e75e40a...,2016-10-02 03:45:00,2016-10-02 03:45:00,2.60,17031062100,17031832600,10.25,2.45,14.70,Credit Card,None,3.349968
8,2d557486f1ebe80551492915d726bef7393ca91d,23f128209b227dfd806428d903d59e15ec4bb57458648d...,2016-03-05 02:45:00,2016-03-05 02:45:00,0.00,17031080300,17031832600,6.75,2.00,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,3.349968
9,2d5cb8290f83470a9fc8da34ac2f10ba2161367b,96a3f43a4eb7400d581a682773569e457cdcba002adf40...,2016-03-14 19:30:00,2016-03-14 20:00:00,4.20,17031320100,17031832600,16.25,3.25,19.50,Credit Card,Dispatch Taxi Affiliation,3.349968


In [12]:
%%time
display(
    tips
    .withColumn('dropoff_census_tract', f.col('dropoff_census_tract').cast(t.StringType()))
    .select(
        f.col('dropoff_census_tract').substr(startPos=6, length=4).alias('spark_block')
    )
    .groupBy('spark_block')
    .count()
    .orderBy('count', ascending=False), 
    10
)

CPU times: user 30 ms, sys: 10 ms, total: 40 ms
Wall time: 5.49 s


,spark_block,count
0,None,348792
1,8391,136992
2,0814,78733
3,3201,78488
4,9800,52747
5,0815,49335
6,0817,47492
7,0818,40612
8,2819,35806
9,0812,34744


In [13]:
split_pd = f.pandas_udf(lambda pd_series: pd_series.str[5:9], returnType=t.StringType())

In [14]:
%%time

display(
    tips
    .withColumn('dropoff_census_tract', f.col('dropoff_census_tract').cast(t.StringType()))
    .select(
        split_pd(f.col('dropoff_census_tract')).alias('pd_block'),
    )
    .groupby('pd_block')
    .count()
    .orderBy('count', ascending=False),
    10
)

CPU times: user 70 ms, sys: 20 ms, total: 90 ms
Wall time: 12.1 s


,pd_block,count
0,None,348792
1,8391,136992
2,0814,78733
3,3201,78488
4,9800,52747
5,0815,49335
6,0817,47492
7,0818,40612
8,2819,35806
9,0812,34744


In [15]:
split_by_row = f.udf(lambda x: None if x is None else x[5:9])

In [16]:
%%time
display(
    tips
    .withColumn('dropoff_census_tract', f.col('dropoff_census_tract').cast(t.StringType()))
    .select(
        split_by_row(f.col('dropoff_census_tract')).alias('udf_block'),
    )
    .groupby('udf_block')
    .count()
    .orderBy('count', ascending=False),
    10
)

CPU times: user 50 ms, sys: 30 ms, total: 80 ms
Wall time: 6.23 s


,udf_block,count
0,None,348792
1,8391,136992
2,0814,78733
3,3201,78488
4,9800,52747
5,0815,49335
6,0817,47492
7,0818,40612
8,2819,35806
9,0812,34744


# UDFs

In [17]:
%%time
tips = tips.withColumn('trip_seconds', (f.unix_timestamp(f.col('end_time')) - f.unix_timestamp(f.col('start_time'))))
display(tips, 5)

CPU times: user 20 ms, sys: 30 ms, total: 50 ms
Wall time: 7.96 s


In [18]:
seconds_pd_udf = f.udf(lambda start_time_series, end_time_series: end_time_series - start_time_series)

In [19]:
%%time
display(
    tips
    .withColumn('trip_seconds', seconds_pd_udf(f.col('start_time'), f.col('end_time'))),
    5
)

CPU times: user 10 ms, sys: 20 ms, total: 30 ms
Wall time: 9.18 s


,trip_id,taxi_id,start_time,end_time,trip_miles,pickup_census_tract,dropoff_census_tract,fare,tips,trip_total,payment_type,company,avg_miles,trip_seconds
0,2d495c6e8ee5cad52f64de2844f5bdd9999c29f5,75dd278c61cc4b3651b76a89407986db8362b1fa6b3947...,2016-07-06 18:45:00,2016-07-06 19:00:00,3.3,17031839100,17031832600,13.25,2.06,15.81,Credit Card,None,3.349968,"Timedelta: 0 days, 900 seconds, 0 microseconds..."
1,2d4c21df0917fd4c630c704b4d1b501c8aec5e14,99ab99695f31d67888669dcd596cbd0cda61f5c717b8d7...,2016-12-16 19:15:00,2016-12-16 19:30:00,2.1,17031081800,17031832600,9.50,1.50,11.50,Credit Card,None,3.349968,"Timedelta: 0 days, 900 seconds, 0 microseconds..."
2,2d4fbb3ea94d4ed30fb45c9c7f278ac75b632fff,6848fb65d081cae1ae77f1828f6ec5fce4bb75c28a81bc...,2016-10-26 16:00:00,2016-10-26 16:15:00,0.1,17031081800,17031832600,12.25,3.05,15.30,Credit Card,Taxi Affiliation Services,3.349968,"Timedelta: 0 days, 900 seconds, 0 microseconds..."
3,2d50340947f1362cdebc258576e8c84423e25f6e,b07a41c369d99e7e7638e698063efe74c8cd8c25e3e9e8...,2016-01-21 20:00:00,2016-01-21 20:00:00,2.7,17031081600,17031832600,10.25,2.55,12.80,Credit Card,Northwest Management LLC,3.349968,"Timedelta: 0 days, 0 seconds, 0 microseconds (..."
4,2d51c476faa582cfa6a4ff99a1e8c08ac2396a0b,9d20d7617e35a8d763ca0bbe3b47308e4bb8e368a8f3bd...,2016-08-12 15:45:00,2016-08-12 17:00:00,15.0,17031980000,17031832600,43.50,14.25,61.75,Credit Card,Northwest Management LLC,3.349968,"Timedelta: 0 days, 4500 seconds, 0 microsecond..."


In [20]:
seconds_udf = f.udf(lambda start_time, end_time: end_time - start_time)

In [21]:
%%time
display(
    tips
    .withColumn('trip_seconds', seconds_udf(f.col('start_time'), f.col('end_time'))),
    5
)

CPU times: user 40 ms, sys: 10 ms, total: 50 ms
Wall time: 11.4 s


,trip_id,taxi_id,start_time,end_time,trip_miles,pickup_census_tract,dropoff_census_tract,fare,tips,trip_total,payment_type,company,avg_miles,trip_seconds
0,2d495c6e8ee5cad52f64de2844f5bdd9999c29f5,75dd278c61cc4b3651b76a89407986db8362b1fa6b3947...,2016-07-06 18:45:00,2016-07-06 19:00:00,3.3,17031839100,17031832600,13.25,2.06,15.81,Credit Card,None,3.349968,"Timedelta: 0 days, 900 seconds, 0 microseconds..."
1,2d4c21df0917fd4c630c704b4d1b501c8aec5e14,99ab99695f31d67888669dcd596cbd0cda61f5c717b8d7...,2016-12-16 19:15:00,2016-12-16 19:30:00,2.1,17031081800,17031832600,9.50,1.50,11.50,Credit Card,None,3.349968,"Timedelta: 0 days, 900 seconds, 0 microseconds..."
2,2d4fbb3ea94d4ed30fb45c9c7f278ac75b632fff,6848fb65d081cae1ae77f1828f6ec5fce4bb75c28a81bc...,2016-10-26 16:00:00,2016-10-26 16:15:00,0.1,17031081800,17031832600,12.25,3.05,15.30,Credit Card,Taxi Affiliation Services,3.349968,"Timedelta: 0 days, 900 seconds, 0 microseconds..."
3,2d50340947f1362cdebc258576e8c84423e25f6e,b07a41c369d99e7e7638e698063efe74c8cd8c25e3e9e8...,2016-01-21 20:00:00,2016-01-21 20:00:00,2.7,17031081600,17031832600,10.25,2.55,12.80,Credit Card,Northwest Management LLC,3.349968,"Timedelta: 0 days, 0 seconds, 0 microseconds (..."
4,2d51c476faa582cfa6a4ff99a1e8c08ac2396a0b,9d20d7617e35a8d763ca0bbe3b47308e4bb8e368a8f3bd...,2016-08-12 15:45:00,2016-08-12 17:00:00,15.0,17031980000,17031832600,43.50,14.25,61.75,Credit Card,Northwest Management LLC,3.349968,"Timedelta: 0 days, 4500 seconds, 0 microsecond..."


In [22]:
tips = tips.withColumn('trip_seconds', (f.unix_timestamp(f.col('end_time')) - f.unix_timestamp(f.col('start_time'))))

# Features

In [28]:
features = (
    tips
    .where(f.col('payment_type') == 'Credit Card')
    .select(
        'trip_id',
        'company',
        f.col('pickup_census_tract').cast(t.DoubleType()).alias('census_tract'),
        f.month('start_time').alias('month'),
        f.dayofweek('start_time').alias('day_of_week'),
        f.hour('start_time').alias('hour'),
        'trip_total',
        'trip_seconds',
        'trip_miles',
        'avg_miles',
        f.col('tips').alias('label'),
    )
)

In [29]:
write_df(features, 'features')